In [0]:
storage_account = 'adworksadlsjc'
client_id = dbutils.secrets.get(scope="aw-adb-kv-scope", key="db-client-id-app")
tenant_id = dbutils.secrets.get(scope="aw-adb-kv-scope", key="db-tenant-id-app")
client_secret = dbutils.secrets.get(scope="aw-adb-kv-scope", key="db-secret-id-app")

spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

### Loading files & creating temporary views

In [0]:
path = "abfss://silver@adworksadlsjc.dfs.core.windows.net/"
df_customer = spark.read.format("delta").load(path + "DimCustomer/")
df_date = spark.read.format("delta").load(path + "DimDate/")
df_geography = spark.read.format("delta").load(path + "DimGeography/")
df_product = spark.read.format("delta").load(path + "DimProduct/")
df_productcategory = spark.read.format("delta").load(path + "DimProductCategory/")
df_productsubcategory = spark.read.format("delta").load(path + "DimProductSubCategory/")
df_sales = spark.read.format("delta").load(path + "FactInternetSales/")

# Creating temporary views
df_customer.createOrReplaceTempView("DimCustomer")
df_date.createOrReplaceTempView("DimDate")
df_geography.createOrReplaceTempView("DimGeography")
df_product.createOrReplaceTempView("DimProduct")
df_productcategory.createOrReplaceTempView("DimProductCategory")
df_productsubcategory.createOrReplaceTempView("DimProductSubCategory")
df_sales.createOrReplaceTempView("FactInternetSales")

In [0]:
# spark.sql("select * from DimCustomer").display()

### DimCustomer

In [0]:
df_dim_customer = spark.sql("""
           SELECT
           CustomerKey,
           FirstName AS First_Name,
           LastName AS Last_Name,
           CONCAT(FirstName, ' ', LastName) AS Full_Name,
           CASE
            WHEN Gender = 'M' THEN 'Male' WHEN Gender = 'F' THEN 'Female' END AS Gender,
           DateFirstPurchase,
           g.city as Customer_City
           FROM
           DimCustomer as c
           LEFT JOIN DimGeography as g
           ON g.GeographyKey = c.GeographyKey
           ORDER BY CustomerKey ASC
            """)

path_gold_cust = "abfss://gold@adworksadlsjc.dfs.core.windows.net/DimCustomer/"
df_dim_customer.write.format("delta").mode("overwrite").save(path_gold_cust)

### DimDate

In [0]:
df_dim_date = spark.sql("""
           SELECT
           DateKey,
           FullDateAlternateKey,
           EnglishDayNameOfWeek AS Day,
           WeekNumberOfYear AS WeekNumber,
           EnglishMonthName AS Month,
           LEFT(EnglishMonthName, 3) AS MonthNumber,
           CalendarQuarter AS Quarter,
           CalendarYear AS Year
           FROM DimDate
                        """)

path_gold_date = "abfss://gold@adworksadlsjc.dfs.core.windows.net/DimDate/"
df_dim_date.write.format("delta").mode("overwrite").save(path_gold_date)

### DimProduct

In [0]:
df_dim_product = spark.sql("""
           SELECT
           p.ProductKey,
           p.ProductAlternateKey AS ProductItemCode,
           p.EnglishProductName AS ProductName,
           ps.EnglishProductSubcategoryName AS SubCategory,
           pc.EnglishProductCategoryName AS ProductCategory,
           p.Color AS ProductColor,
           p.Size AS ProductSize,
           p.ProductLine AS ProductLine,
           p.ModelName AS ProductModelName,
           p.EnglishDescription AS ProductDescription,
           COALESCE(p.Status, 'Outdated') AS ProductStatus
           FROM DimProduct as p
           LEFT JOIN DimProductSubcategory AS ps
           ON ps.ProductSubcategoryKey = p.ProductSubcategoryKey
           LEFT JOIN DimProductCategory AS pc
           ON pc.ProductCategoryKey = ps.ProductCategoryKey
           ORDER BY p.ProductKey ASC
                        """)

path_gold_product = "abfss://gold@adworksadlsjc.dfs.core.windows.net/DimProduct/"
df_dim_product.write.format("delta").mode("overwrite").save(path_gold_product)

%md
### FactInternetSales

In [0]:
df_fact_sales = spark.sql("""
           SELECT
           ProductKey,
           OrderDateKey,
           DueDateKey,
           ShipDateKey,
           CustomerKey,
           SalesOrderNumber,
           OrderDate,
           DueDate,
           ShipDate,
           SalesAmount
           FROM FactInternetSales
           ORDER BY OrderDateKey ASC
                        """)

path_gold_sales = "abfss://gold@adworksadlsjc.dfs.core.windows.net/FactInterntetSales/"
df_fact_sales.write.format("delta").mode("overwrite").save(path_gold_sales)